# The Python concurrency story

## Concurrency

[dictionary.com](http://www.dictionary.com/browse/concurrent)
- occurring or existing simultaneously or side by side
- acting in conjunction; cooperating

### The free lunch is over
- approaching the physical limits of Moore's Law.
- processors not getting faster
- way forward is to have MORE processors, not faster processors


[The free lunch is over](http://www.gotw.ca/publications/concurrency-ddj.htm)

### cocurrency is the way forward
- allows for scaling 'horizontally'
- explains the sudden surge of interest in Erlang, Scala, Clojure and functional programming.

### concurrency allows for some cool things
- better resource utilization
- handle web scale and big data
- applications that are responsive, that feel real time
GUI, games
- model the real world

### Waiters and tables
- 5 tables
- 1 waiter: who can take one order at a time
- Customers(Table) think about the order for 4 seconds
- Waiter takes down order in 1 seconds


In [ ]:
# sequential_waiter.py
import time
from datetime import datetime

class Waiter(object):

    def __init__(self, name):
        self.name = name
    
    def take_order(self):
        time.sleep(1)

class Table(object):

    def __init__(self, name):
        self.order_taken = False
        self.name = name
    
    def give_order(self, waiter):
        print(f"{self.name} is thinking about the order")
        time.sleep(4)
        self.order_taken = True
        waiter.take_order()
        print(f"{self.name} has given it's order to {waiter.name}")

tables = [Table("A"), Table("B"), Table("C"), Table("D"), Table("E")]
waiter = Waiter("John Doe")

start_time = datetime.now()
for table in tables:
    table.give_order(waiter)
end_time = datetime.now()
elapsed_time = end_time - start_time
print(f"taking all orders took {elapsed_time.seconds} seconds")


In [ ]:
# multithreaded example
import threading
import time
from datetime import datetime

class Waiter(object):
    lock = threading.Lock()
    def __init__(self, name):
        self.name = name
    
    def take_order(self):
        with self.lock:  # TODO: queue example if adequate time
        # waiter can take only one order at a time.
        # Sleep does not block the entire Python process
        # here context switch will happen to another thread
        # so we have to lock so that the waiter isn't
        # allowed to be at two tables at once
            time.sleep(1)

class Table(object):

    def __init__(self, name):
        self.order_taken = False
        self.name = name
    
    def give_order(self, waiter):
        print(f"{self.name} is thinking about the order")
        time.sleep(4)
        self.order_taken = True
        waiter.take_order()
        print(f"{self.name} has given it's order to {waiter.name}")

tables = [Table("A"), Table("B"), Table("C"), Table("D"), Table("E")]
waiter = Waiter("John Doe")

start_time = datetime.now()

threads = []
for table in tables:
    thread = threading.Thread(target=table.give_order, args=(waiter,))
    # all tables start thinking about ordering
    thread.start()
    threads.append(thread)

for thread in threads:
    thread.join()  # wait for all threads to finish
    # otherwise execution will continue without waiting for threads
    # to end, and we'll get elapsed time as 0.

end_time = datetime.now()
elapsed_time = end_time - start_time
print(f"took {elapsed_time.seconds} seconds")

## Concurrency is not parallelism

__concurrency__: Dealing with a lot at once, property of the solution (code)

__parallelism__: Doing a lot at once, property of the runtime

### Why you should care
- lets you to pick the right tool for the right job
- python threads don't run parallely, you can still do concurrency.

## Multithreading in python

__Advantages__
- excellent for speeding up blocking I/O bound programs
- Scheduled preemptively
- Lighter memory footprint than processes
- Shared memory - faster communication

__Disdavantages__
- GIL means that we can't take advantages of multiple cores.
- Context switch may happen whenever, have to synchronize access to critical section.
- Hard to test and debug
- Context switches are hardly free. Can see degradation in performance.

In [1]:
# sequential_client.py
import requests
from datetime import datetime

start_time = datetime.now()
for i in range(5):
    print(requests.get("http://127.0.0.1:5000").text)
end_time = datetime.now()
print(f"elapsed time {(end_time - start_time).seconds} s")

Hello, World!, I am thread Thread-1
Hello, World!, I am thread Thread-2
Hello, World!, I am thread Thread-3
Hello, World!, I am thread Thread-4
Hello, World!, I am thread Thread-5
elapsed time 5 s


In [ ]:
# threaded_client.py
import threading
import datetime
from datetime import datetime

start_time = datetime.now()

def make_request():
    print(requests.get("http://127.0.0.1:5000").text)

threads = []    
for i in range(5):
    thread = threading.Thread(target=make_request)
    thread.start()
    threads.append(thread)

for thread in threads:
    thread.join()

end_time = datetime.now()
elapsed_time = end_time - start_time
print(f"took {elapsed_time.seconds} seconds")



### GIL the good parts
- Python has one global lock, on the entire interpreter instead of thousands of granular locks everywhere else.
- GIL makes it easy to integrate non thread safe C libraries
- Locking and unlocking is not free, previous attempts at removing the GIL has resulted in severe degradation of the performance of a single thread.

## Python's approach to parallelism - Don't DIY
- To make use of all cores, python prescribes using multiple processes.
- Utilizing multiple cores is left up to the OS (since processes are scheduled by the OS).



### Multiprocessing


__Advantages__:
- Can workaround the GIL by spawning multiple processes.
- That makes them better at CPU bound tasks
- Crashing processes will not kill our entire program

__Disadvantages__:
- Require IPC to communicate between processes.
- slower context switches.
- more startup cost.

In [ ]:
# cpu_bound_sequential.py
from datetime import datetime

def cpu_bound_task():
    [i for i in range(100000)]
    return True

start_time = datetime.now()
print(all([cpu_bound_task() for i in range(1000)]))
end_time = datetime.now()
elapsed_time = end_time - start_time
print(f"took {elapsed_time}")

In [ ]:
# cpu_bound_multithreaded.py
from datetime import datetime
from multiprocessing.pool import ThreadPool

p = ThreadPool(4)
def cpu_bound_task(*args):
    [i for i in range(100000)]
    return True
start_time = datetime.now()
print(all(p.map(cpu_bound_task, [i for i in range(1000)], 100)))
p.close()
p.join()
end_time = datetime.now()
elapsed_time = end_time - start_time
print(f"took {elapsed_time}")

In [ ]:
# cpu_bound_multiprocess.py
from multiprocessing import Pool
p = Pool(4)
def cpu_bound_task(*args):
    [i for i in range(100000)]
    return True
start_time = datetime.now()
print(all(p.map(cpu_bound_task, [i for i in range(1000)], 100)))
p.close()
p.join()
end_time = datetime.now()
elapsed_time = end_time - start_time
print(f"took {elapsed_time}")
    

## Async
- cooperative scheduling - need explicit code to cause task switch.
- Typically single threaded
- Since you control when task switches occur, you don't need locks for synchronization.


__Advantages__:
- very low cost, since no context switch. Incidentally they're cheaper than function calls. Cheaper switching mechanism among all techniques. People prefer this model to locking, 
- No cost of synchronization = less CPU consumption. Async servers > threaded servers. You can run many many more async tasks than threads.

__disadvantages__:
- Need code that gives up control
- Need an event loop
- Everything has to be non-blocking.
- Need to learn a lot of new things - new syntax, new libraries(aysnc versions), eventloops, futures.

In [ ]:
# async example
import asyncio
import time
from datetime import datetime

class Waiter(object):

    def __init__(self, name):
        self.name = name
    
    def take_order(self):
        # blocks thread, hence blocking the entire event loop.
        time.sleep(1)

class Table(object):

    def __init__(self, name):
        self.order_taken = False
        self.name = name

    # async keyword defines a coroutine
    async def give_order(self, waiter):
        print(f"{self.name} is thinking about the order")
        await asyncio.sleep(4)  # This is how we give up control
        # Execution will resume after 4 s.
        self.order_taken = True
        waiter.take_order()
        print(f"{self.name} has given it's order to {waiter.name}")

tables = [Table("A"), Table("B"), Table("C"), Table("D"), Table("E")]
waiter = Waiter("John Doe")

eventloop = asyncio.get_event_loop()
start_time = datetime.now()
tasks = [eventloop.create_task(table.give_order(waiter))
         for table in tables]
await asyncio.wait(tasks)
end_time = datetime.now()
elapsed_time = end_time - start_time
print(f"taking all orders took {elapsed_time.seconds} seconds")


In [ ]:
#async_client.py
import aiohttp
import asyncio
from datetime import datetime

loop = asyncio.get_event_loop()

async def make_request(url):
    async with aiohttp.request('get', url) as resp:
        print(await resp.read())



tasks = [
    loop.create_task(
        make_request("http://127.0.0.1:5000")
    ) for _ in range(5)
]

start_time = datetime.now()
await asyncio.wait(tasks)
end_time = datetime.now()
elapsed_time = end_time - start_time
print(f"took {elapsed_time.seconds} seconds")

## Recapping
- Sync: Blocking operations.
- Concurrency: Making progress together.
- Async: Non blocking operations.
- Parallelism: Making progress in parallel.
- locking: avoid

## Conclusion
| feature/ library | threading | multiprocessing | asyncio |
|------------------------|-----------------------------------------|-------------------|--------------------------------------------------|
| scheduling | preemptive | preemptive | cooperative |
| use multiple CPUs | no | yes | no |
| scalability | medium (100s) | low (10s) | high (1000s) |
| use blocking functions | yes | yes | no |
| when to use | Existing codebase, I/O bound, blocking calls | CPU bound tasks |New codebase, I/O bound, large scale |

## Links

- [python concurrency story](https://powerfulpython.com/blog/python-concurrency-story-pt1/)
- [Python concurrency keynote - Raymond Hettinger](https://www.youtube.com/watch?v=9zinZmE3Ogk)
[Python Concurrency From the Ground Up - David Beazley](https://www.youtube.com/watch?v=MCs5OvhV9S4)
- [github repo for the talk](https://github.com/madhukar93/python_concurrency)